In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [3]:
data_path = "Data/features_3_sec.csv"
df_main = pd.read_csv(data_path)

In [4]:
df_main.shape
X = df_main.drop('label',axis=1)
y = df_main['label']

In [5]:
#encoding the labels for future referencing
index_map = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
genre_map = {'blues':0,
             'classical':1,
            'country': 2,
            'disco': 3,
            'hiphop':4,
            'jazz':5,
            'metal':6,
            'pop':7,
            'reggae':8,
            'rock':9}

for i in range(len(y)):
    y[i] = genre_map[y[i]]
    
y = y.astype(int)

C:\Users\caleb\AppData\Local\Temp\ipykernel_9128\4226296203.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i] = genre_map[y[i]]


In [6]:
X = X.drop(['filename','length','harmony_mean','harmony_var','perceptr_mean','perceptr_var'],axis = 1)
X.shape

(9990, 53)

# Train Test Split

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_rem, y_train, y_rem = train_test_split(X,y,stratify = y, test_size = 0.15)
X_test, X_val, y_test, y_val = train_test_split(X_rem, y_rem , stratify = y_rem , test_size = 0.5)
print(X_train.shape )
print(X_val.shape)

(8491, 53)
(750, 53)


In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)
X_val = sc.transform(X_val)

# SVM Classifier

In [9]:
from sklearn.svm import SVC
SVM_classifier = SVC(kernel = 'poly',C =0.1,gamma = 1)
SVM_classifier.fit(X_train,y_train)

SVC(C=0.1, gamma=1, kernel='poly')

In [10]:
y_pred = SVM_classifier.predict(X_val)
y_pred

array([6, 9, 1, 7, 3, 8, 4, 0, 4, 4, 4, 0, 2, 5, 7, 1, 4, 9, 6, 7, 3, 9,
       3, 0, 7, 2, 9, 3, 7, 1, 2, 7, 9, 7, 8, 5, 2, 2, 3, 6, 1, 3, 3, 9,
       5, 4, 7, 5, 8, 9, 1, 7, 0, 4, 5, 3, 7, 8, 0, 0, 7, 4, 9, 0, 0, 9,
       8, 3, 0, 2, 8, 7, 6, 5, 9, 8, 0, 4, 6, 3, 6, 0, 5, 2, 3, 4, 7, 2,
       5, 9, 7, 8, 6, 7, 8, 3, 8, 0, 8, 4, 6, 5, 4, 1, 0, 5, 5, 8, 0, 1,
       3, 0, 5, 6, 6, 7, 0, 0, 4, 9, 6, 9, 3, 2, 5, 7, 2, 4, 0, 8, 5, 7,
       6, 9, 2, 5, 4, 0, 3, 4, 2, 3, 6, 9, 6, 5, 4, 5, 1, 1, 0, 5, 4, 8,
       8, 0, 0, 0, 4, 4, 2, 6, 5, 6, 0, 6, 3, 4, 2, 8, 5, 9, 6, 2, 8, 2,
       7, 1, 2, 2, 3, 8, 8, 8, 9, 1, 0, 7, 9, 1, 2, 3, 0, 6, 5, 3, 8, 1,
       7, 6, 5, 3, 5, 5, 3, 1, 1, 1, 7, 8, 7, 8, 9, 2, 5, 5, 1, 1, 0, 4,
       5, 1, 2, 5, 2, 6, 8, 6, 4, 2, 1, 6, 7, 7, 2, 9, 0, 4, 4, 7, 0, 3,
       9, 2, 3, 1, 6, 7, 8, 6, 9, 6, 0, 2, 2, 0, 7, 2, 2, 6, 4, 6, 1, 8,
       3, 1, 2, 5, 3, 3, 4, 4, 6, 0, 4, 7, 3, 1, 9, 4, 4, 5, 4, 2, 3, 9,
       8, 3, 1, 8, 4, 7, 9, 7, 7, 6, 0, 8, 3, 0, 8,

In [11]:
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
cm = confusion_matrix(y_val,y_pred)
print(cm)
print('accuracy score: ', accuracy_score(y_val,y_pred))
print('f1_score : ', f1_score(y_val,y_pred,average='micro'))

accuracy score:  0.8533333333333334
f1_score :  0.8533333333333335


# GridSearchCV

In [12]:
from sklearn.model_selection import GridSearchCV

#defining parameter range
param_grid = {'C' : [0.1,1,10],
             'gamma': [1, 0.1],
             'kernel': ['rbf','poly']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
history_svm = grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.141 total time=  14.9s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.181 total time=  14.8s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.143 total time=  14.7s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.157 total time=  14.7s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.164 total time=  14.5s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.865 total time=   3.7s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.872 total time=   3.9s
[CV 3/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.859 total time=   3.9s
[CV 4/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.879 total time=   3.9s
[CV 5/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.863 total time=   3.9s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.465 total time=  12.9s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;

In [13]:
# print best parameter after tuning
print(grid.best_params_)
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 0.1, 'gamma': 0.1, 'kernel': 'poly'}
SVC(C=0.1, gamma=0.1, kernel='poly')


In [14]:
grid_predictions = grid.predict(X_val)
print('accuracy score: ', accuracy_score(y_val,grid_predictions))
print('f1_score : ', f1_score(y_val,grid_predictions,average='micro'))

accuracy score:  0.8586666666666667
f1_score :  0.8586666666666666


# Metadata Generator

In [15]:
import librosa
def getdataf(filename):
    y, sr = librosa.load(filename)
    #fetching tempo
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    #fetching beats
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,sr=sr)
    #chroma_stft
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    #rmse
    rmse = librosa.feature.rms(y=y)
    #fetching spectral centroid
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    #spectral bandwidth
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    #fetching spectral rolloff
    spec_rolloff = librosa.feature.spectral_rolloff(y=y+0.01, sr=sr)[0]
    #zero crossing rate
    zero_crossing = librosa.feature.zero_crossing_rate(y)
    #mfcc
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    #metadata dictionary
    metadata_dict = {'chroma_stft_mean':np.mean(chroma_stft),'chroma_stft_var':np.var(chroma_stft),'rmse_mean':np.mean(rmse),'rmse_var':np.var(rmse),
                     'spectral_centroid_mean':np.mean(spec_centroid),'spectral_centroid_var':np.var(spec_centroid),'spectral_bandwidth_mean':np.mean(spec_bw),'spectral_bandwidth':np.var(spec_bw), 
                     'rolloff_mean':np.mean(spec_rolloff),'rolloff_var':np.var(spec_rolloff), 'zero_crossing_rate_mean':np.mean(zero_crossing),'zero_crossing_rate_var':np.var(zero_crossing),
                    # 'harmony_mean':np.mean(t_harmonics),'harmony_var':np.var(t_harmonics),
                    # 'perceptr_mean':np.mean(perceptual_CQT),'perceptr_var':np.var(perceptual_CQT),
                     'tempo':tempo}

    for i in range(1,21):
        metadata_dict.update({'mfcc'+str(i)+'_mean':np.mean(mfcc[i-1])})
        metadata_dict.update({'mfcc'+str(i)+'_var':np.var(mfcc[i-1])})
    df_meta = pd.DataFrame(metadata_dict,index=[0])
    return df_meta

In [16]:
#Prediction Accuracy after hyperparameter tuning
grid_predictions_test = grid.predict(X_test)
print('accuracy score: ', accuracy_score(y_test,grid_predictions_test))
print('f1_score : ', f1_score(y_test,grid_predictions_test,average='micro'))


accuracy score:  0.8891855807743658
f1_score :  0.8891855807743658


# XGBoost

In [17]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [18]:
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

In [19]:
def plotTraining(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training_acc')
    plt.plot(epochs, val_acc, 'b', label='Validation_acc')
    plt.title('Training and validation accuracy')
    plt.legend() 

    plt.figure() 

    plt.plot(epochs, loss, 'bo', label='Training_loss')
    plt.plot(epochs, val_loss, 'b', label='Validation_loss')
    plt.title('Training and validation loss')
    plt.legend() 

    plt.show()

In [20]:
model_xgboost = xgboost.XGBClassifier(learning_rate=0.1,
                                      max_depth=5,
                                      n_estimators=5000,
                                      subsample=0.5,
                                      colsample_bytree=0.5,
                                      eval_metric='auc',
                                      verbosity=1)

eval_set = [(X_val, y_val)]

model_xgboost.fit(X_train,
                  y_train,
                  early_stopping_rounds=10,
                  eval_set=eval_set,
                  verbose=True)

[0]	validation_0-auc:0.85928


C:\Users\caleb\anaconda3\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1]	validation_0-auc:0.89573
[2]	validation_0-auc:0.91451
[3]	validation_0-auc:0.92408
[4]	validation_0-auc:0.93299
[5]	validation_0-auc:0.93799
[6]	validation_0-auc:0.94117
[7]	validation_0-auc:0.94302
[8]	validation_0-auc:0.94530
[9]	validation_0-auc:0.94622
[10]	validation_0-auc:0.94801
[11]	validation_0-auc:0.94922
[12]	validation_0-auc:0.94973
[13]	validation_0-auc:0.95110
[14]	validation_0-auc:0.95326
[15]	validation_0-auc:0.95390
[16]	validation_0-auc:0.95573
[17]	validation_0-auc:0.95659
[18]	validation_0-auc:0.95770
[19]	validation_0-auc:0.95858
[20]	validation_0-auc:0.95914
[21]	validation_0-auc:0.95996
[22]	validation_0-auc:0.96062
[23]	validation_0-auc:0.96108
[24]	validation_0-auc:0.96162
[25]	validation_0-auc:0.96194
[26]	validation_0-auc:0.96245
[27]	validation_0-auc:0.96278
[28]	validation_0-auc:0.96280
[29]	validation_0-auc:0.96311
[30]	validation_0-auc:0.96323
[31]	validation_0-auc:0.96368
[32]	validation_0-auc:0.96424
[33]	validation_0-auc:0.96472
[34]	validation_0-a

[269]	validation_0-auc:0.98675
[270]	validation_0-auc:0.98678
[271]	validation_0-auc:0.98673
[272]	validation_0-auc:0.98677
[273]	validation_0-auc:0.98676
[274]	validation_0-auc:0.98674
[275]	validation_0-auc:0.98671
[276]	validation_0-auc:0.98673
[277]	validation_0-auc:0.98673
[278]	validation_0-auc:0.98673
[279]	validation_0-auc:0.98674
[280]	validation_0-auc:0.98677


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=5000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

# Testing on New song

In [26]:
#df genration for new song
df=getdataf("1000.wav")
df = sc.transform(df)
print(df)
test_predict = grid.predict(df)
print(test_predict)

C:\Users\caleb\AppData\Local\Temp\ipykernel_9128\1568602305.py:10: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)


[[-1.54265197  0.1160742  -0.16720028  1.82888448 -1.59653387 -0.5586302
  -1.79962935  0.53395641 -1.78050598 -0.75147617 -1.02959767  0.14149852
   0.82953494 -0.88962147  3.56717154  1.25716062  1.74311451  0.27575491
   0.30072578 -1.41139713  0.49839751  0.92330465  0.21194254 -1.7868914
   0.30961823 -0.16976013  0.42981296 -1.67248043  0.52809704 -0.58552272
   0.14132031 -1.48047036  0.90338973 -0.33135825  0.52676692 -1.63656536
   0.66955668 -0.1360401   0.47610572 -0.85734593  0.42253159  0.37700214
   0.2936702  -0.50392676  1.13090324  0.3150548   0.76616234  0.05838287
   0.54691871 -0.89459962  1.00183481 -0.61658653  0.86116736]]
[2]


C:\Users\caleb\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- rmse_mean
- rmse_var
- spectral_bandwidth
Feature names seen at fit time, yet now missing:
- rms_mean
- rms_var
- spectral_bandwidth_var

  warnings.warn(message, FutureWarning)


In [27]:
test_predict1 = model_xgboost.predict(df)
print(test_predict1)
print(list(genre_map.keys())[list(genre_map.values()).index(test_predict1)])

[2]
country
